
<p style="text-align:center"> <img src="https://i.ibb.co/NKMg9s3/62gx34.jpg">

<br>

# <center> YOLOX + YOLOv5 Weighted Boxes Fusion Ensemble 2.o</center>

<br>



### How to ensemble object detection models?
This notebook shows how to detect starfish objects (COTS dataset) using multiple models on Kaggle (YOLOX & YOLOv5 for example). 

<hr>

**Original Notebooks Credits:**

Do upvote this notebooks: 

Original Notebook: [Great-Barrier-Reef: YOLOX + YOLOv5 Ensemble](https://www.kaggle.com/yamqwe/great-barrier-reef-yolox-yolov5-ensemble)

I ensemble the models from [YoloX full training pipeline for COTS dataset](https://www.kaggle.com/remekkinas/yolox-full-training-pipeline-for-cots-dataset) and [Great-Barrier-Reef: YOLOv5 [infer]](https://www.kaggle.com/awsaf49/great-barrier-reef-yolov5-infer).

<hr>

## Goal of this notebook:
### Fixing the overlapping issue in orignal notebook and improve the score
<p style="text-align:center"> <img src="https://i.ibb.co/7njx8SH/Be-Funky-collage.jpg" width="1000" height="500">


In [ ]:
import warnings; warnings.filterwarnings("ignore")

import os
import cv2
import ast
import sys
import glob
import torch
import shutil
import importlib
import traceback
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
sys.path.append('../input/tensorflow-great-barrier-reef')
tqdm.pandas()

from PIL import Image
from IPython.display import display

### Installing YOLOX 
<div class="alert alert-warning" role="alert"><strong>Some Kaggle enviroment hacking :) due to competition limitation - no internet access during submission.</strong></div>

In [ ]:
%cp -r /kaggle/input/yolox-cots-models /kaggle/working/
%cd /kaggle/working/yolox-cots-models/yolox-dep

In [ ]:
# Install YOLOX required modules
!pip install pip-21.3.1-py3-none-any.whl -f ./ --no-index
!pip install loguru-0.5.3-py3-none-any.whl -f ./ --no-index
!pip install ninja-1.10.2.3-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl -f ./ --no-index
!pip install onnx-1.8.1-cp37-cp37m-manylinux2010_x86_64.whl -f ./ --no-index
!pip install onnxruntime-1.8.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl -f ./ --no-index
!pip install onnxoptimizer-0.2.6-cp37-cp37m-manylinux2014_x86_64.whl -f ./ --no-index
!pip install thop-0.0.31.post2005241907-py3-none-any.whl -f ./ --no-index
!pip install tabulate-0.8.9-py3-none-any.whl -f ./ --no-index

In [ ]:
# Install YOLOX
%cd /kaggle/working/yolox-cots-models/YOLOX
!pip install -r requirements.txt
!pip install -v -e . 

In [ ]:
# Install CocoAPI tool
%cd /kaggle/working/yolox-cots-models/yolox-dep/cocoapi/PythonAPI
!make
!make install
!python setup.py install

In [ ]:
import pycocotools

### Test Model

In [ ]:
%cd /kaggle/working/yolox-cots-models/YOLOX

CHECKPOINT_FILE = '/kaggle/working/yolox-cots-models/yx_l_003.pth'

In [ ]:
config_file_template = '''

#!/usr/bin/env python3
# -*- coding:utf-8 -*-
# Copyright (c) Megvii, Inc. and its affiliates.

import os

from yolox.exp import Exp as MyExp


class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        self.depth = 1
        self.width = 1
        self.exp_name = os.path.split(os.path.realpath(__file__))[1].split(".")[0]
        self.num_classes = 1

'''

with open('cots_config.py', 'w') as f:
    f.write(config_file_template)

In [ ]:
from yolox.utils import postprocess
from yolox.data.data_augment import ValTransform

COCO_CLASSES = (
  "starfish",
)

# get YOLOX experiment
current_exp = importlib.import_module('cots_config')
exp = current_exp.Exp()

# set inference parameters
test_size = (800, 1280)
num_classes = 1
confthre = 0.2
nmsthre = 0.2


# get YOLOX model
yolox_model = exp.get_model()
yolox_model.cuda()
yolox_model.eval()

# get custom trained checkpoint
ckpt_file = CHECKPOINT_FILE
ckpt = torch.load(ckpt_file, map_location="cpu")
yolox_model.load_state_dict(ckpt["model"])

In [ ]:
def yolox_inference(img, model, test_size): 
    bboxes = []
    bbclasses = []
    scores = []
    
    preproc = ValTransform(legacy = False)

    tensor_img, _ = preproc(img, None, test_size)
    tensor_img = torch.from_numpy(tensor_img).unsqueeze(0)
    tensor_img = tensor_img.float()
    tensor_img = tensor_img.cuda()

    with torch.no_grad():
        outputs = model(tensor_img)
        outputs = postprocess(
                    outputs, num_classes, confthre,
                    nmsthre, class_agnostic=True
                )

    if outputs[0] is None:
        return [], [], []
    
    outputs = outputs[0].cpu()
    bboxes = outputs[:, 0:4]

    bboxes /= min(test_size[0] / img.shape[0], test_size[1] / img.shape[1])
    bbclasses = outputs[:, 6]
    scores = outputs[:, 4] * outputs[:, 5]
    
    return bboxes, bbclasses, scores

In [ ]:
def draw_yolox_predictions(img, bboxes, scores, bbclasses, confthre, classes_dict):
    for i in range(len(bboxes)):
            box = bboxes[i]
            cls_id = int(bbclasses[i])
            score = scores[i]
            if score < confthre:
                continue
            x0 = int(box[0])
            y0 = int(box[1])
            x1 = int(box[2])
            y1 = int(box[3])

            cv2.rectangle(img, (x0, y0), (x1, y1), (0, 255, 0), 2)
            cv2.putText(img, '{}:{:.1f}%'.format(classes_dict[cls_id], score * 100), (x0, y0 - 3), cv2.FONT_HERSHEY_PLAIN, 0.8, (0,255,0), thickness = 1)
    return img

# YOLOv5 Model

In [ ]:
CKPT_PATH = '/kaggle/input/reef-baseline-fold12/l6_3600_uflip_vm5_f12_up/f1/best.pt'
IMG_SIZE  = 9000
CONF      = 0.35
IOU       = 0.20
AUGMENT   = True

In [ ]:
def get_path(row):
    row['image_path'] = f'{ROOT_DIR}/train_images/video_{row.video_id}/{row.video_frame}.jpg'
    return row

In [ ]:
# Train Data
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
df = pd.read_csv(f'{ROOT_DIR}/train.csv')
df = df.progress_apply(get_path, axis=1)
df['annotations'] = df['annotations'].progress_apply(lambda x: ast.literal_eval(x))

In [ ]:
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
data = (df.num_bbox>0).value_counts()/len(df)*100

In [ ]:
def voc2yolo(bboxes, image_height=720, image_width=1280):
    """
    voc  => [x1, y1, x2, y1]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]/ image_height
    
    w = bboxes[..., 2] - bboxes[..., 0]
    h = bboxes[..., 3] - bboxes[..., 1]
    
    bboxes[..., 0] = bboxes[..., 0] + w/2
    bboxes[..., 1] = bboxes[..., 1] + h/2
    bboxes[..., 2] = w
    bboxes[..., 3] = h
    
    return bboxes

def yolo2voc(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

def coco2yolo(bboxes, image_height=720, image_width=1280):
    """
    coco => [xmin, ymin, w, h]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # normolizinig
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    
    # converstion (xmin, ymin) => (xmid, ymid)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    
    return bboxes

def yolo2coco(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    coco => [xmin, ymin, w, h]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # denormalizing
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    
    # converstion (xmid, ymid) => (xmin, ymin) 
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    
    return bboxes

def voc2coco(bboxes, image_height=720, image_width=1280):
    bboxes  = voc2yolo(bboxes, image_height, image_width)
    bboxes  = yolo2coco(bboxes, image_height, image_width)
    return bboxes

def coco2voc(bboxes, image_height=720, image_width=1280):
    bboxes  = coco2yolo(bboxes, image_height, image_width)
    bboxes  = yolo2voc(bboxes, image_height, image_width)
    return bboxes

def load_image(image_path):
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)


def plot_one_box(x, img, color=None, label=None, line_thickness=None,score=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    label=label+"{:.2f}%".format(score)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def draw_bboxes(img, bboxes, classes, class_ids, colors = None, show_classes = None, bbox_format = 'yolo', class_name = False, line_thickness = 2,scores=None):  
     
    image = img.copy()
    show_classes = classes if show_classes is None else show_classes
    colors = (0, 255 ,0) if colors is None else colors
    
    if bbox_format == 'yolo':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            score   = scores[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes:
            
                x1 = round(float(bbox[0])*image.shape[1])
                y1 = round(float(bbox[1])*image.shape[0])
                w  = round(float(bbox[2])*image.shape[1]/2) #w/2 
                h  = round(float(bbox[3])*image.shape[0]/2)

                voc_bbox = (x1-w, y1-h, x1+w, y1+h)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(get_label(cls)),
                             line_thickness = line_thickness,score=score)
            
    elif bbox_format == 'coco':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            score   = scores[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes:            
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                w  = int(round(bbox[2]))
                h  = int(round(bbox[3]))

                voc_bbox = (x1, y1, x1+w, y1+h)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness,score=score)

    elif bbox_format == 'voc_pascal':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            score   = scores[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes: 
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                x2 = int(round(bbox[2]))
                y2 = int(round(bbox[3]))
                voc_bbox = (x1, y1, x2, y2)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness,score=score)
    else:
        raise ValueError('wrong bbox format')

    return image

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row

np.random.seed(32)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [ ]:
def predict(model, img, size=768, augment=False):
    height, width = img.shape[:2]
    results = model(img, size=size, augment=augment)  # custom inference size
    preds   = results.pandas().xyxy[0]
    bboxes  = preds[['xmin','ymin','xmax','ymax']].values
    if len(bboxes):
        bboxes  = voc2coco(bboxes,height,width).astype(int)
        confs   = preds.confidence.values
        return bboxes, confs
    else:
        return [],[]

def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            conf             = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot


def show_img(img, bboxes, bbox_format='yolo',scores=None):
    names  = ['starfish']*len(bboxes)
    labels = [0]*len(bboxes)
    img    = draw_bboxes(img = img,
                           bboxes = bboxes, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = bbox_format,
                           line_thickness = 2,scores=scores)
    return Image.fromarray(img).resize((800, 400))

In [ ]:
def load_model(ckpt_path, conf=0.15, iou=0.2):
    model = torch.hub.load('/kaggle/input/yolov5-lib-ds','custom',path=ckpt_path,
                           source='local',force_reload=True)  # local repo
    model.conf = conf  # NMS confidence threshold
    model.iou  = iou  # NMS IoU threshold
    model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 1000  # maximum number of detections per image
    return model

# Ensembling

## Let's look at some train images!

In [ ]:
import sys; sys.path.append('/kaggle/input/weightedboxesfusion/')
# !pip install --no-deps '/kaggle/input/weightedboxesfusion/' > /dev/null

In [ ]:
TEST_IMAGE_PATH = "/kaggle/input/tensorflow-great-barrier-reef/train_images/video_0/9674.jpg"
#import torchvision.ops.boxes as bops
from ensemble_boxes import *

def run_wbf(bboxes, confs, image_size=1280, iou_thr=0.2, skip_box_thr=0.001, weights=None):
    boxes =  [bbox/(image_size) for bbox in bboxes]
    
    scores = [conf for conf in confs]
    labels = [np.ones(conf.shape[0]) for conf in confs]
    
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=[1,1], iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    boxes = boxes*(image_size-1)
    boxes=voc2coco(boxes).astype(int)
    #print("voc:coco",boxes)
    return boxes, scores, labels

model = load_model(CKPT_PATH, conf=CONF, iou=IOU)
image_paths = df[df.num_bbox>1].sample(200).image_path.tolist()
for idx, path in enumerate(image_paths):
    img = cv2.imread(path)[...,::-1]
    
    bboxes_1, bbclasses, scores = yolox_inference(img[...,::-1], yolox_model, test_size)        
    bboxes_2, confis = predict(model, img, size=IMG_SIZE, augment=AUGMENT)        
    #pred_1, pred_2 = voc2coco(bboxes_1.detach().numpy()).astype(int), bboxes_2
    
    if len(bboxes_1) > 0 and len(bboxes_2) > 0: 
        pred_1, pred_2 = bboxes_1.detach().numpy(), coco2voc(bboxes_2).astype(int)
        boxes, scores1, labels = run_wbf([pred_1, pred_2], [scores, confis], image_size = 1280)
    elif len(bboxes_1) > 0: boxes, scores1 = bboxes_1.detach().numpy(), scores
    elif len(bboxes_2) > 0: boxes, scores1 = voc2coco(bboxes_2,img.shape[1],img.shape[2]).astype(int), confis
    
    if len(bboxes_1) > 0:            
        print('\n\nYOLOX Predictions: ')
        display(show_img(img, voc2coco(bboxes_1.detach().numpy(),img.shape[1],img.shape[2]).astype(int), bbox_format='coco',scores=scores))
    else:        
        print('\n\nYOLOX Predictions: ')
        display(show_img(img, [], bbox_format='coco',scores=scores))
    if len(bboxes_2) > 0:
        print('\n\nYoloV5 Predictions: ')
        display(show_img(img, bboxes_2, bbox_format='coco',scores=confis))
    else:            
        print('\n\nYOLOV5 Predictions: ')
        display(show_img(img, [], bbox_format='coco',scores=confis))
    if len(bboxes_1) > 0 and len(bboxes_2) > 0: 
        print('\n\nEnsemble (WBF) Predictions: ')
        display(show_img(img, boxes, bbox_format='coco',scores=scores1))
    else:            
        print('\n\nEnsemble (WBF) Predictions: ')
        display(show_img(img, [], bbox_format='coco',scores=scores1))
    
    if idx>5:
        break

## SUBMIT PREDICTION TO COMPETITION

In [ ]:
%cd /kaggle/working/

In [ ]:
import greatbarrierreef

env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()  

In [ ]:
submission_dict = {
    'id': [],
    'prediction_string': [],
}

for (image_np, sample_prediction_df) in iter_test:
    bboxes_1, bbclasses, scores = yolox_inference(image_np[...,::-1], yolox_model, test_size)
    bboxes_2, confis = predict(model, image_np, size=IMG_SIZE, augment=AUGMENT)        

    if len(bboxes_1) > 0 and len(bboxes_2) > 0: 
        pred_1, pred_2 = bboxes_1.detach().numpy(), coco2voc(bboxes_2).astype(int)
        boxes, scores1, labels = run_wbf([pred_1, pred_2], [scores, confis], image_size = 1280)
    elif len(bboxes_1) > 0: boxes, scores1 = voc2coco(bboxes_1.detach().numpy()).astype(int), scores
    elif len(bboxes_2) > 0: boxes, scores1 = bboxes_2, confis
    else: boxes = []
    # display(show_img(image_np, bboxes, bbox_format='coco'))
    predictions = []
    for i in range(len(boxes)):
        box = boxes[i]        
        score = scores1[i]
        if score > 0.1:
            x_min = int(box[0])
            y_min = int(box[1])
            bbox_width = int(box[2])
            bbox_height = int(box[3])
            predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))

    prediction_str = ' '.join(predictions)
    sample_prediction_df['annotations'] = prediction_str
    env.predict(sample_prediction_df)

    print('Prediction:', prediction_str)

In [ ]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()

<p style="font-size:130%;text-align:center"> This notebook is just bug fixing ⚙🛠!</p>
<p style="font-size:220%;text-align:center"> If you find this notebook useful, please do Upvote to Original Notebook 🙏:) </p>

# <p style="text-align:center"> <img src="https://media.giphy.com/media/3oEdva9BUHPIs2SkGk/giphy.gif"> </p>

<p style="font-size:130%;text-align:center"> Tip: Add tracking to same notebook will boost score!... </p>

<hr>

**Original Notebooks Credits:**

[Great-Barrier-Reef: YOLOX + YOLOv5 Ensemble](https://www.kaggle.com/yamqwe/great-barrier-reef-yolox-yolov5-ensemble)

[YoloX full training pipeline for COTS dataset](https://www.kaggle.com/remekkinas/yolox-full-training-pipeline-for-cots-dataset)

[Great-Barrier-Reef: YOLOv5 [infer]](https://www.kaggle.com/awsaf49/great-barrier-reef-yolov5-infer).

<hr>